In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('/home/sacsresta/Documents/RESEARCH/Project/sentiment/merged_data_AAPL_from_2015-01-01_to_2025-03-01.csv')
df['shifted_direction'] = df['Direction'].shift(-1)
df = df.drop(columns=['Supertrend','UpperBand', 'LowerBand', 'Uptrend',
       'Downtrend', 'headline','Adj Close'])
df['Date'] = pd.to_datetime(df['Date'])
df.dropna(inplace=True)
print(df.columns)

Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'Direction',
       'sentiment_score', 'shifted_direction'],
      dtype='object')


In [2]:
import pandas as pd

X = df[['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 
       'sentiment_score', 'shifted_direction']].copy()
def create_data(X, max_lag = 60):
  # Define the number of lags you want
  max_lag = max_lag  # For example, to create lags 1 through 5

  # Create lagged features in a loop
  for i in range(1, max_lag + 1):
      X[f'Close_lag{i}'] = X['Close'].shift(i)

  # Define X and y (drop the first two rows with NaN lags)
  y = X['shifted_direction'].iloc[max_lag:].astype(int)
  X = X.drop(columns = ['shifted_direction']).iloc[max_lag:]
  return X,y

X,y = create_data(X)


In [3]:
X.shape,y.shape

((2494, 67), (2494,))

In [4]:
X.isnull().sum()

Date           0
Close          0
High           0
Low            0
Open           0
              ..
Close_lag56    0
Close_lag57    0
Close_lag58    0
Close_lag59    0
Close_lag60    0
Length: 67, dtype: int64

In [5]:
y.isnull().sum()

np.int64(0)

In [6]:
y.value_counts()

shifted_direction
 1    1492
-1    1002
Name: count, dtype: int64

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
encoder = LabelEncoder()

In [9]:
y = encoder.fit_transform(y)

In [10]:
X.set_index('Date',inplace=True)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)

X_train.shape,X_test.shape,y_train.shape,y_test.shape

((1745, 66), (749, 66), (1745,), (749,))

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier


# Updated classifier dictionary
classifiers = {
    'Logistic Regression': LogisticRegression(C=10, max_iter=1000, solver='liblinear'),
    'Random Forest': RandomForestClassifier(max_depth=10, min_samples_split=10, n_estimators=50),
    'XGBoost': XGBClassifier(learning_rate=0.01, max_depth=3, n_estimators=100, eval_metric='logloss'),
    'SVM': SVC(C=10, kernel='linear', probability=True),
    'Naive Bayes': BernoulliNB(),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3),  # Uses decision trees [[7]]
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),  # Instance-based learning
    'Decision Tree': DecisionTreeClassifier(max_depth=5, min_samples_split=10),  # Base tree model [[7]]
    'AdaBoost': AdaBoostClassifier(n_estimators=50, learning_rate=0.5),  # Boosting ensemble
    'SGD Classifier': SGDClassifier(loss='log_loss', alpha=0.001),  # Linear model with SGD [[10]]
    'MLP': MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000),  # Neural network
    'LGBMClassifier' :LGBMClassifier(
    n_estimators=100, 
    learning_rate=0.05, 
    max_depth=5, 
    boosting_type='gbdt'
)
}
# Create list to store results
results = []

# Train-test loop
for name, clf in classifiers.items():
    # Train model
    clf.fit(X_train_scaled, y_train)

    # Make predictions
    y_pred = clf.predict(X_test_scaled)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    # Store results
    results.append((name, accuracy, cm))

    # Display results
    print(f"\n{name} Performance:")
    print(f"Accuracy: {accuracy:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print(report)



Logistic Regression Performance:
Accuracy: 0.8879
Confusion Matrix:
[[310  45]
 [ 39 355]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       355
           1       0.89      0.90      0.89       394

    accuracy                           0.89       749
   macro avg       0.89      0.89      0.89       749
weighted avg       0.89      0.89      0.89       749


Random Forest Performance:
Accuracy: 0.6275
Confusion Matrix:
[[229 126]
 [153 241]]
              precision    recall  f1-score   support

           0       0.60      0.65      0.62       355
           1       0.66      0.61      0.63       394

    accuracy                           0.63       749
   macro avg       0.63      0.63      0.63       749
weighted avg       0.63      0.63      0.63       749


XGBoost Performance:
Accuracy: 0.5394
Confusion Matrix:
[[ 10 345]
 [  0 394]]
              precision    recall  f1-score   support

           0       1.00      0.03

/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f


Gradient Boosting Performance:
Accuracy: 0.5781
Confusion Matrix:
[[251 104]
 [212 182]]
              precision    recall  f1-score   support

           0       0.54      0.71      0.61       355
           1       0.64      0.46      0.54       394

    accuracy                           0.58       749
   macro avg       0.59      0.58      0.57       749
weighted avg       0.59      0.58      0.57       749


K-Nearest Neighbors Performance:
Accuracy: 0.6649
Confusion Matrix:
[[217 138]
 [113 281]]
              precision    recall  f1-score   support

           0       0.66      0.61      0.63       355
           1       0.67      0.71      0.69       394

    accuracy                           0.66       749
   macro avg       0.66      0.66      0.66       749
weighted avg       0.66      0.66      0.66       749


Decision Tree Performance:
Accuracy: 0.5794
Confusion Matrix:
[[ 59 296]
 [ 19 375]]
              precision    recall  f1-score   support

           0       0.76

/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [21]:
for i, k in classifiers.items():
    print(k)

LogisticRegression(C=10, max_iter=1000, solver='liblinear')
RandomForestClassifier(max_depth=10, min_samples_split=10, n_estimators=50)
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)
SVC(C=10, kernel='linear', probability=True)
BernoulliNB()
GradientBoostingClassifier()


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define hyperparameter grids
param_grids = {
    'Logistic Regression': {
        'C': [0.1, 1, 10], 
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga']
    },
    'Random Forest': {
        'n_estimators': [50, 100],
        'max_depth': [5, 10, None],
        'min_samples_split': [2, 10],
        'max_features': ['sqrt', 'log2']
    },
    'XGBoost': {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5],
        'n_estimators': [100, 200],
        'subsample': [0.6, 0.8],
        'colsample_bytree': [0.6, 1.0]
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf'],
        'gamma': ['scale', 'auto']
    },
    'Naive Bayes': {
        'alpha': [0.1, 0.5, 1.0]
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.05, 0.1],
        'max_depth': [3, 5],
        'min_samples_split': [2, 10]
    },
    'K-Nearest Neighbors': {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree'],
        'p': [1, 2]
    },
    'Decision Tree': {
        'criterion': ['gini', 'entropy'],
        'max_depth': [5, 10, None],
        'min_samples_split': [2, 10],
        'max_features': [None, 'sqrt']
    },
    'AdaBoost': {
        'n_estimators': [50, 100],
        'learning_rate': [0.1, 0.5, 1.0]
    },
    'SGD Classifier': {
        'alpha': [0.0001, 0.001],
        'loss': ['log_loss', 'modified_huber'],
        'penalty': ['l2', 'l1', 'elasticnet'],
        'learning_rate': ['constant', 'optimal']
    },
    'MLP': {
        'hidden_layer_sizes': [(100,), (50,50)],
        'activation': ['relu', 'tanh'],
        'solver': ['adam', 'sgd'],
        'learning_rate_init': [0.001, 0.01],
        'alpha': [0.0001, 0.001]
    }
}

# Dictionary to store best models
best_models = {}

for name, clf in classifiers.items():
    print(f"Optimizing {name}...")
    
    if name in param_grids:
        grid_search = GridSearchCV(clf, param_grids[name], cv=5, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train_scaled, y_train)
        best_models[name] = grid_search.best_estimator_  # Save best model
        print(f"Best parameters for {name}: {grid_search.best_params_}")
    else:
        clf.fit(X_train_scaled, y_train)
        best_models[name] = clf
print(f"No hyperparameters to tune for {name}")

Optimizing Logistic Regression...


Best parameters for Logistic Regression: {'C': 10, 'solver': 'liblinear'}
Optimizing Random Forest...
Best parameters for Random Forest: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 50}
Optimizing XGBoost...
Best parameters for XGBoost: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Optimizing SVM...
Best parameters for SVM: {'C': 10, 'kernel': 'linear'}
Optimizing Naive Bayes...
Best parameters for Naive Bayes: {}
Optimizing Gradient Boosting...
Optimizing K-Nearest Neighbors...
Optimizing Decision Tree...
Optimizing AdaBoost...
Optimizing SGD Classifier...
Optimizing MLP...
Optimizing LGBMClassifier...
[LightGBM] [Info] Number of positive: 1098, number of negative: 647
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000980 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16830
[LightGBM] [Info] Number of data points in the train set: 1745, number of used features: 66
[Lig

In [23]:
best_models

{'Logistic Regression': LogisticRegression(C=10, max_iter=1000, solver='liblinear'),
 'Random Forest': RandomForestClassifier(max_depth=10, n_estimators=50),
 'XGBoost': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric='logloss',
               feature_types=None, feature_weights=None, gamma=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.01, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=3, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=100, n_jobs=None,
               num_parallel_tree=None, ...),
 'SVM': SVC(C=10, kernel='linear', probabi

# Hyperparameter tuning

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# Create list to store results
results = []

# Train-test loop
for name, clf in best_models.items():
    # Train model
    clf.fit(X_train_scaled, y_train)

    # Make predictions
    y_pred = clf.predict(X_test_scaled)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    # Store results
    results.append((name, accuracy, cm))

    # Display results
    print(f"\n{name} Performance:")
    print(f"Accuracy: {accuracy:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print(report)



Logistic Regression Performance:
Accuracy: 0.8879
Confusion Matrix:
[[310  45]
 [ 39 355]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       355
           1       0.89      0.90      0.89       394

    accuracy                           0.89       749
   macro avg       0.89      0.89      0.89       749
weighted avg       0.89      0.89      0.89       749


Random Forest Performance:
Accuracy: 0.6088
Confusion Matrix:
[[255 100]
 [193 201]]
              precision    recall  f1-score   support

           0       0.57      0.72      0.64       355
           1       0.67      0.51      0.58       394

    accuracy                           0.61       749
   macro avg       0.62      0.61      0.61       749
weighted avg       0.62      0.61      0.61       749


XGBoost Performance:
Accuracy: 0.5394
Confusion Matrix:
[[ 10 345]
 [  0 394]]
              precision    recall  f1-score   support

           0       1.00      0.03

/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f


Gradient Boosting Performance:
Accuracy: 0.5808
Confusion Matrix:
[[252 103]
 [211 183]]
              precision    recall  f1-score   support

           0       0.54      0.71      0.62       355
           1       0.64      0.46      0.54       394

    accuracy                           0.58       749
   macro avg       0.59      0.59      0.58       749
weighted avg       0.59      0.58      0.58       749


K-Nearest Neighbors Performance:
Accuracy: 0.6649
Confusion Matrix:
[[217 138]
 [113 281]]
              precision    recall  f1-score   support

           0       0.66      0.61      0.63       355
           1       0.67      0.71      0.69       394

    accuracy                           0.66       749
   macro avg       0.66      0.66      0.66       749
weighted avg       0.66      0.66      0.66       749


Decision Tree Performance:
Accuracy: 0.5794
Confusion Matrix:
[[ 59 296]
 [ 19 375]]
              precision    recall  f1-score   support

           0       0.76

/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [25]:
from sklearn.feature_selection import SelectKBest, f_classif

# Select top 10 features
selector = SelectKBest(f_classif, k=10)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

# Updated classifier dictionary
classifiers = {
    'Logistic Regression': LogisticRegression(C=10, max_iter=1000, solver='liblinear'),
    'Random Forest': RandomForestClassifier(max_depth=10, min_samples_split=10, n_estimators=50),
    'XGBoost': XGBClassifier(learning_rate=0.01, max_depth=3, n_estimators=100, eval_metric='logloss'),
    'SVM': SVC(C=10, kernel='linear', probability=True),
    'Naive Bayes': BernoulliNB(),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3),  # Uses decision trees [[7]]
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),  # Instance-based learning
    'Decision Tree': DecisionTreeClassifier(max_depth=5, min_samples_split=10),  # Base tree model [[7]]
    'AdaBoost': AdaBoostClassifier(n_estimators=50, learning_rate=0.5),  # Boosting ensemble
    'SGD Classifier': SGDClassifier(loss='log_loss', alpha=0.001),  # Linear model with SGD [[10]]
    'MLP': MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)  # Neural network
}
# Create list to store results
results = []

# Train-test loop
for name, clf in classifiers.items():
    # Train model
    clf.fit(X_train_selected, y_train)

    # Make predictions
    y_pred = clf.predict(X_test_selected)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    # Store results
    results.append((name, accuracy, cm))

    # Display results
    print(f"\n{name} Performance:")
    print(f"Accuracy: {accuracy:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print(report)



Logistic Regression Performance:
Accuracy: 0.6822
Confusion Matrix:
[[136 219]
 [ 19 375]]
              precision    recall  f1-score   support

           0       0.88      0.38      0.53       355
           1       0.63      0.95      0.76       394

    accuracy                           0.68       749
   macro avg       0.75      0.67      0.65       749
weighted avg       0.75      0.68      0.65       749


Random Forest Performance:
Accuracy: 0.5554
Confusion Matrix:
[[ 42 313]
 [ 20 374]]
              precision    recall  f1-score   support

           0       0.68      0.12      0.20       355
           1       0.54      0.95      0.69       394

    accuracy                           0.56       749
   macro avg       0.61      0.53      0.45       749
weighted avg       0.61      0.56      0.46       749



/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f


XGBoost Performance:
Accuracy: 0.5260
Confusion Matrix:
[[  0 355]
 [  0 394]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       355
           1       0.53      1.00      0.69       394

    accuracy                           0.53       749
   macro avg       0.26      0.50      0.34       749
weighted avg       0.28      0.53      0.36       749


SVM Performance:
Accuracy: 0.6822
Confusion Matrix:
[[140 215]
 [ 23 371]]
              precision    recall  f1-score   support

           0       0.86      0.39      0.54       355
           1       0.63      0.94      0.76       394

    accuracy                           0.68       749
   macro avg       0.75      0.67      0.65       749
weighted avg       0.74      0.68      0.65       749


Naive Bayes Performance:
Accuracy: 0.5260
Confusion Matrix:
[[  0 355]
 [  0 394]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       3

/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f


Gradient Boosting Performance:
Accuracy: 0.5768
Confusion Matrix:
[[ 57 298]
 [ 19 375]]
              precision    recall  f1-score   support

           0       0.75      0.16      0.26       355
           1       0.56      0.95      0.70       394

    accuracy                           0.58       749
   macro avg       0.65      0.56      0.48       749
weighted avg       0.65      0.58      0.50       749


K-Nearest Neighbors Performance:
Accuracy: 0.5567
Confusion Matrix:
[[ 56 299]
 [ 33 361]]
              precision    recall  f1-score   support

           0       0.63      0.16      0.25       355
           1       0.55      0.92      0.69       394

    accuracy                           0.56       749
   macro avg       0.59      0.54      0.47       749
weighted avg       0.59      0.56      0.48       749


Decision Tree Performance:
Accuracy: 0.5314
Confusion Matrix:
[[  4 351]
 [  0 394]]
              precision    recall  f1-score   support

           0       1.00

In [27]:
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [28]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)


In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Define classifiers with corrected hyperparameters
classifiers = {
    'Logistic Regression': LogisticRegression(C=10, max_iter=1000, solver='liblinear'),
    'Random Forest': RandomForestClassifier(max_depth=10, min_samples_split=10, n_estimators=50),
    'XGBoost': XGBClassifier(learning_rate=0.01, max_depth=3, n_estimators=100, eval_metric='logloss'),
    'SVM': SVC(C=10, kernel='linear', probability=True),
    'Naive Bayes': BernoulliNB()
}

# Create list to store results
results = []

# Train-test loop
for name, clf in classifiers.items():
    # Train model
    clf.fit(X_train_resampled, y_train_resampled)

    # Make predictions
    y_pred = clf.predict(X_test_scaled)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    # Store results
    results.append((name, accuracy, cm))

    # Display results
    print(f"\n{name} Performance:")
    print(f"Accuracy: {accuracy:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print(report)



Logistic Regression Performance:
Accuracy: 0.8838
Confusion Matrix:
[[316  39]
 [ 48 346]]
              precision    recall  f1-score   support

           0       0.87      0.89      0.88       355
           1       0.90      0.88      0.89       394

    accuracy                           0.88       749
   macro avg       0.88      0.88      0.88       749
weighted avg       0.88      0.88      0.88       749


Random Forest Performance:
Accuracy: 0.6449
Confusion Matrix:
[[113 242]
 [ 24 370]]
              precision    recall  f1-score   support

           0       0.82      0.32      0.46       355
           1       0.60      0.94      0.74       394

    accuracy                           0.64       749
   macro avg       0.71      0.63      0.60       749
weighted avg       0.71      0.64      0.60       749


XGBoost Performance:
Accuracy: 0.6061
Confusion Matrix:
[[ 81 274]
 [ 21 373]]
              precision    recall  f1-score   support

           0       0.79      0.23

/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [31]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(estimators=[
    ('Random Forest', best_models['Random Forest']),
    ('Logistic Regression', best_models['Logistic Regression']),
    ('SVM', best_models['SVM'])
], voting='hard')  # 'soft' for probability averaging

voting_clf.fit(X_train_scaled, y_train)


VotingClassifier(estimators=[('Random Forest',
                              RandomForestClassifier(max_depth=10,
                                                     n_estimators=50)),
                             ('Logistic Regression',
                              LogisticRegression(C=10, max_iter=1000,
                                                 solver='liblinear')),
                             ('SVM',
                              SVC(C=10, kernel='linear', probability=True))])

In [32]:
y_pred = voting_clf.predict(X_test_scaled)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)


In [33]:
print(f" Performance:")
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(cm)

 Performance:
Accuracy: 0.8892
Confusion Matrix:
[[317  38]
 [ 45 349]]


In [34]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

for name, model in best_models.items():
    y_pred = model.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print(f"\n{name} (Optimized) Performance:")
    print(f"Accuracy: {accuracy:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print(classification_report(y_test, y_pred))



Logistic Regression (Optimized) Performance:
Accuracy: 0.8879
Confusion Matrix:
[[310  45]
 [ 39 355]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       355
           1       0.89      0.90      0.89       394

    accuracy                           0.89       749
   macro avg       0.89      0.89      0.89       749
weighted avg       0.89      0.89      0.89       749


Random Forest (Optimized) Performance:
Accuracy: 0.6088
Confusion Matrix:
[[255 100]
 [193 201]]
              precision    recall  f1-score   support

           0       0.57      0.72      0.64       355
           1       0.67      0.51      0.58       394

    accuracy                           0.61       749
   macro avg       0.62      0.61      0.61       749
weighted avg       0.62      0.61      0.61       749


XGBoost (Optimized) Performance:
Accuracy: 0.5394
Confusion Matrix:
[[ 10 345]
 [  0 394]]
              precision    recall  f1-score   suppor

/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [35]:
y_train = y_train.reshape(-1, 1)

In [36]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import keras.backend as K
K.clear_session()

model = Sequential([
    Dense(128, activation='relu'),
    Dropout(0.3),  # Dropout to reduce overfitting
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Multi-class: softmax, Binary: sigmoid
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
# Train the model
history = model.fit(X_train_scaled, y_train, 
                    epochs=50, batch_size=32, 
                    validation_split=0.2, verbose=1)



2025-03-27 21:26:48.345444: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743071208.418259  227806 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743071208.439288  227806 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743071208.607224  227806 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743071208.607249  227806 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743071208.607252  227806 computation_placer.cc:177] computation placer alr

Epoch 1/50


2025-03-27 21:26:50.920343: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-03-27 21:26:50.920408: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-03-27 21:26:50.920414: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: Sachin
2025-03-27 21:26:50.920417: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:190] hostname: Sachin
2025-03-27 21:26:50.920519: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:197] libcuda reported version is: 550.120.0
2025-03-27 21:26:50.920533: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:201] kernel reported version is: 550.120.0
2025-03

44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6081 - loss: 0.6695 - val_accuracy: 0.6476 - val_loss: 0.5836
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7144 - loss: 0.5777 - val_accuracy: 0.3553 - val_loss: 1.2483
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7236 - loss: 0.5533 - val_accuracy: 0.7794 - val_loss: 0.5255
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7659 - loss: 0.4909 - val_accuracy: 0.5330 - val_loss: 0.8610
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7554 - loss: 0.4823 - val_accuracy: 0.5244 - val_loss: 0.9973
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7777 - loss: 0.4790 - val_accuracy: 0.7135 - val_loss: 0.4598
Epoch 7/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7649 - loss: 0.4812 - val_accuracy: 0.3610 - val_loss: 1.8828
Epoch 8/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7981 - loss: 0.4300 - val_accuracy: 0.4756 - val_loss: 0.9993
Epo

In [50]:
# First get raw probabilities for precision threshold tuning
y_probs = model.predict(X_test_scaled)

# Apply threshold optimized via validation data (replace with your method from [[2]] [[7]])
optimal_threshold = 0.63  # Example from validation analysis in search result [[2]]
y_pred = y_probs > optimal_threshold

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 996us/step


In [51]:
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(cm)
print(classification_report(y_test, y_pred))

Accuracy: 0.6368
Confusion Matrix:
[[ 87 268]
 [  4 390]]
              precision    recall  f1-score   support

           0       0.49      1.00      0.66       355
           1       1.00      0.06      0.12       394

    accuracy                           0.51       749
   macro avg       0.75      0.53      0.39       749
weighted avg       0.76      0.51      0.37       749



In [ ]:
import numpy as np

print("X_train_scaled shape:", X_train_scaled.shape)
print("y_train_encoded shape:", y_train_encoded.shape)

print("Missing values in X_train_scaled:", np.isnan(X_train_scaled).sum())
print("Missing values in y_train_encoded:", np.isnan(y_train_encoded).sum() if num_classes > 2 else "N/A")


X_train_scaled shape: (1745, 66)
y_train_encoded shape: (1745,)
Missing values in X_train_scaled: 0
Missing values in y_train_encoded: N/A


In [ ]:
X_train_scaled.shape

(1745, 66)